In [1]:
pip install transformers torch

                                              0.0/7.2 MB ? eta -:--:--
Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\HP\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 435, in _error_catcher
    yield
  File "C:\Users\HP\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 516, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\HP\anaconda3\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "C:\Users\HP\anaconda3\lib\http\client.py", line 463, in read
    n = self.readinto(b)
  File "C:\Users\HP\anaconda3\lib\http\client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\HP\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\HP\anaconda3\lib\ssl.py", line 1242, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\HP\anaconda3\lib\ssl.py", line 1100, in read
    return self._sslobj.read(len, buffer)
socket.timeout: The read op

# BERT

In [10]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def compare_notes(notes1, notes2):
    # Load pre-trained BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Tokenize and encode the notes
    tokens1 = tokenizer.encode(notes1, add_special_tokens=True)
    tokens2 = tokenizer.encode(notes2, add_special_tokens=True)

    # Convert token IDs to tensors
    input_ids1 = torch.tensor(tokens1).unsqueeze(0)
    input_ids2 = torch.tensor(tokens2).unsqueeze(0)

    # Generate embeddings using BERT model
    with torch.no_grad():
        outputs1 = model(input_ids1)
        embeddings1 = outputs1.last_hidden_state[:, 0, :].numpy()

        outputs2 = model(input_ids2)
        embeddings2 = outputs2.last_hidden_state[:, 0, :].numpy()

    # Compute cosine similarity between embeddings
    similarity = cosine_similarity(embeddings1, embeddings2)[0][0]

    return similarity

# Example usage
notes1 = "C:\\Users\\HP\\Desktop\\New Text Document (2).txt"
notes2 = "C:\\Users\\HP\\Desktop\\notes.pdf"

similarity_score = compare_notes(notes1, notes2)
print(f"Similarity score: {similarity_score}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Similarity score: 0.931411623954773


# TF IDF

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import PyPDF2

def compare_notes(notes1, notes2):
    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the notes into TF-IDF vectors
    tfidf_matrix = vectorizer.fit_transform([notes1, notes2])

    # Calculate the cosine similarity between the TF-IDF vectors
    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

    return similarity

with open("C:\\Users\\HP\\Desktop\\New Text Document (2).txt", 'r', encoding='utf-8') as file:
    notes1 = file.read()
  
with open("C:\\Users\\HP\\Desktop\\notes.pdf", 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    notes2 = ""
    for page in reader.pages:
        notes2 = page.extract_text()

similarity_score = compare_notes(notes1, notes2)
print(f"Similarity score: {similarity_score}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\HP\\Desktop\\New Text Document (2).txt'

# UI TF IDF

In [3]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import PyPDF2

def compare_notes(notes1, notes2):
    vectorizer = TfidfVectorizer()

    tfidf_matrix = vectorizer.fit_transform([notes1, notes2])

    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

    return similarity

def open_text_file():
    file_path = filedialog.askopenfilename(filetypes=[("Text Files", "*.txt")])
    if file_path:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
        text_entry.delete("1.0", tk.END)
        text_entry.insert(tk.END, text)

def open_pdf_file():
    file_path = filedialog.askopenfilename(filetypes=[("PDF Files", "*.pdf")])
    if file_path:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
        compare_entry.delete("1.0", tk.END)
        compare_entry.insert(tk.END, text)

def compare():
    notes1 = text_entry.get("1.0", tk.END)
    notes2 = compare_entry.get("1.0", tk.END)
    similarity_score = compare_notes(notes1, notes2)
    similarity_label.config(text=f"Similarity score: {similarity_score:.2f}")

# Create the main window
window = tk.Tk()
window.title("Notes Comparison")
window.geometry("500x400")


style = ttk.Style()
style.configure("TLabel", foreground="#333", font=("Arial", 12))
style.configure("TButton", font=("Arial", 12))
style.configure("TText", font=("Arial", 12))

# First Set of Notes
text_label = ttk.Label(window, text="First Set of Notes:")
text_label.pack()

text_entry = tk.Text(window, height=5)
text_entry.pack(pady=5)

open_text_button = ttk.Button(window, text="Open Text File", command=open_text_file)
open_text_button.pack()

# Second Set of Notes
compare_label = ttk.Label(window, text="Second Set of Notes:")
compare_label.pack()

compare_entry = tk.Text(window, height=5)
compare_entry.pack(pady=5)

open_pdf_button = ttk.Button(window, text="Open PDF File", command=open_pdf_file)
open_pdf_button.pack()

# Compare Button
compare_button = ttk.Button(window, text="Compare Notes", command=compare)
compare_button.pack(pady=10)

# Similarity Score
similarity_label = ttk.Label(window, text="Similarity score: ")
similarity_label.pack()

# Run the main event loop
window.mainloop()


# NLP

In [2]:
from nltk.corpus import stopwords 
from nltk import download 
download('stopwords')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
import PyPDF2
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def preprocess_text(text):
    # Tokenize the text into individual words
    tokens = nltk.word_tokenize(text)
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    # Remove stopwords and punctuation
    stopwords_set = set(stopwords.words("english"))
    tokens = [token for token in tokens if token.isalnum() and token not in stopwords_set]
    # Join the tokens back into a single string
    preprocessed_text = " ".join(tokens)
    return preprocessed_text

def compare_answers(student_answer, ideal_answer):
    # Preprocess the student and ideal answers
    notes1 = preprocess_text(student_answer)
    notes2 = preprocess_text(ideal_answer)

    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()
    # Fit the vectorizer on the preprocessed answers
    vectors = vectorizer.fit_transform([notes1, notes2])

    # Compute the cosine similarity between the vectors
    similarity = cosine_similarity(vectors[0], vectors[1])[0][0]
    return similarity

def open_text_file():
    file_path = filedialog.askopenfilename(filetypes=[("Text Files", "*.txt")])
    if file_path:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
        text_entry.delete("1.0", tk.END)
        text_entry.insert(tk.END, text)

def open_pdf_file():
    file_path = filedialog.askopenfilename(filetypes=[("PDF Files", "*.pdf")])
    if file_path:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
        compare_entry.delete("1.0", tk.END)
        compare_entry.insert(tk.END, text)
        
def compare():
    notes1 = text_entry.get("1.0", tk.END)
    notes2 = compare_entry.get("1.0", tk.END)
    similarity_score = compare_notes(notes1, notes2)
    similarity_label.config(text=f"Similarity score: {similarity_score:.2f}")    
    
    
# Create the main window
window = tk.Tk()
window.title("Notes Comparison")
window.geometry("700x400")


style = ttk.Style()
style.configure("TLabel", foreground="#333", font=("Arial", 12))
style.configure("TButton", font=("Arial", 12))
style.configure("TText", font=("Arial", 12))

# First Set of Notes
text_label = ttk.Label(window, text="First Set of Notes:")
text_label.pack()

text_entry = tk.Text(window, height=5)
text_entry.pack(pady=5)

open_text_button = ttk.Button(window, text="Open Text File", command=open_text_file)
open_text_button.pack()

# Second Set of Notes
compare_label = ttk.Label(window, text="Second Set of Notes:")
compare_label.pack()

compare_entry = tk.Text(window, height=5)
compare_entry.pack(pady=5)

open_pdf_button = ttk.Button(window, text="Open PDF File", command=open_pdf_file)
open_pdf_button.pack()

# Compare Button
compare_button = ttk.Button(window, text="Compare Notes", command=compare)
compare_button.pack(pady=10)

# Similarity Score
similarity_label = ttk.Label(window, text="Similarity score: ")
similarity_label.pack()

# Run the main event loop
window.mainloop()
